In [1]:
import os
import sys
import io
import json
import operator
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import nltk
from google.cloud import speech
from google.cloud import storage
from google.cloud import language_v1
from nltk.corpus import stopwords
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.compositing.concatenate import concatenate_videoclips
import nltk
import pdb
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from keras_preprocessing.sequence import pad_sequences
from nltk.sentiment import SentimentIntensityAnalyzer

import spacy
import moviepy.editor as mp
from datetime import timedelta
import moviepy.editor as mp
from pydub import AudioSegment, silence
import translation

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "creds/celtic-guru-247118-9e8cccc27c4a.json"
audio_path = 'temp_audio.wav'
bucket_name = 'sermon-speech-audio'

In [4]:
    client = speech.SpeechClient()
    speech_audio = speech.RecognitionAudio(uri='gs://sermon-speech-audio/temp_audio.wav')
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code='en-US',
        audio_channel_count = 2,
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True,
        model='video',
        use_enhanced=True)

In [5]:
operation = client.long_running_recognize(config=config, audio=speech_audio)

In [128]:
print("test")

test


In [7]:
response = operation.result(timeout=5400)

In [8]:
response

results {
  alternatives {
    transcript: "Take your copy of God\'s word this morning. Please find Esther chapter number 7 The Book of Esther chapter number 7. Every great story has an antagonist it has a villain who is either working behind the scenes unknown to most of the characters in the story or in some situations he is working openly and what he\'s doing is known to all of the characters in the story."
    confidence: 0.912838221
    words {
      start_time {
        nanos: 300000000
      }
      end_time {
        nanos: 500000000
      }
      word: "Take"
    }
    words {
      start_time {
        nanos: 500000000
      }
      end_time {
        nanos: 600000000
      }
      word: "your"
    }
    words {
      start_time {
        nanos: 600000000
      }
      end_time {
        nanos: 800000000
      }
      word: "copy"
    }
    words {
      start_time {
        nanos: 800000000
      }
      end_time {
        nanos: 900000000
      }
      word: "of"
    }
    

In [82]:
words = []

words.append({
    "word": word,
    "metadata": {
      "author": "Bard",
      "date": "2023-06-09",
      "topic": "Natural Language Processing"
    }
  })

nlp.add_pipe(words[0])

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got {'word': Take, 'metadata': {'author': 'Bard', 'date': '2023-06-09', 'topic': 'Natural Language Processing'}} (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

In [130]:
sentences = []
for result in response.results:
    alternatives = result.alternatives
    
    for alternative in alternatives:
        sentence = alternative.transcript.strip()
        if sentence:
            sentences.append({
                'text': sentence,
                'start_time': alternative.words[0].start_time.total_seconds(),
                'end_time': alternative.words[-1].end_time.total_seconds()
            })

sentences

[{'text': "Take your copy of God's word this morning. Please find Esther chapter number 7 The Book of Esther chapter number 7. Every great story has an antagonist it has a villain who is either working behind the scenes unknown to most of the characters in the story or in some situations he is working openly and what he's doing is known to all of the characters in the story.",
  'start_time': 0.3,
  'end_time': 29.9},
 {'text': "Tory. Now if you're like me, whenever you watch a good movie or you read a good story, you try to figure out who the villain is as quickly as possible, and then when you figure out who the villain is, you then spend the rest of your time, hoping for and routing on the people to expose the enemy and make him or her pay for their misdeeds. And if the story is really, really good, then you cheer when Justice is finally",
  'start_time': 30.0,
  'end_time': 59.9},
 {'text': "Fenced, it doesn't happen very often in movies. But back a while ago, I went and saw a seri

In [127]:
import nltk.data

def is_end_of_sentence(text):
    # Load the NLTK sentence tokenizer
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    # Tokenize the text into sentences
    test_sentences = tokenizer.tokenize(text)

    # Check if the last sentence is complete
    is_sentence_end = test_sentences[-1].endswith(('.', '!', '?'))
    print(f"detecting {test_sentences[-1]} didn't end {text}")
    return is_sentence_end

# Example usage
text = "Find if you would Esther chapter 5, the fifth chapter of the Book of Esther. You know, some might look at the story of Esther and see only a series of coincidences. Well, you know, it's just a coincidence that the king in a drunken stupor would call out to the queen vashti and insist that she present herself in front of himself and other drunk men in a way that we had degrade the office of Queen. And therefore she said that she would not come and she was eventually deposed because of that action, which then opened the door for a new Queen. We might look and say, well, you know, it's just a coincidence that Mordecai and Esther remained and Susa it's just a coincidence that Esther would have found favor in the eyes of those to whom she was responsible and that she would eventually find"
result = is_end_of_sentence(text)
print(result)


detecting We might look and say, well, you know, it's just a coincidence that Mordecai and Esther remained and Susa it's just a coincidence that Esther would have found favor in the eyes of those to whom she was responsible and that she would eventually find didn't end Find if you would Esther chapter 5, the fifth chapter of the Book of Esther. You know, some might look at the story of Esther and see only a series of coincidences. Well, you know, it's just a coincidence that the king in a drunken stupor would call out to the queen vashti and insist that she present herself in front of himself and other drunk men in a way that we had degrade the office of Queen. And therefore she said that she would not come and she was eventually deposed because of that action, which then opened the door for a new Queen. We might look and say, well, you know, it's just a coincidence that Mordecai and Esther remained and Susa it's just a coincidence that Esther would have found favor in the eyes of thos

In [126]:
new_sentences = []

# Iterate over the sentences
i = 0
while i + 1 < len(sentences):
    new_sentence = {}
    current_sentence = sentences[i]
    current_text = current_sentence['text']
    current_start_time = current_sentence['start_time']
    current_end_time = current_sentence['end_time']

    next_sentence = sentences[i + 1]
    next_text = next_sentence['text']
    next_start_time = next_sentence['start_time']
    next_end_time = next_sentence['end_time']

    # Process the current sentence text with spaCy
    current_doc = nlp(current_text)
    next_doc = nlp(next_text)

    # Check if the last token is the end of a sentence
    #print(doc[-1])
    if not current_doc[-1].text.endswith(('.', '!', '?')):
        print("detecting nonend" + current_doc[-1].text)
        # Concatenate with the next sentence if it exists
        if i + 1 < len(sentences):
            # Concatenate the texts, update the end time
            current_text += ' ' + next_text
            new_sentence = {'text': current_text, 'start_time': current_start_time, 'end_time': next_end_time}
            i += 1 # skip next sentence
            new_sentences.append(new_sentence)
    else:
        new_sentences.append(current_sentence)

    # Move to the next sentence
    i += 1

new_sentences


detecting nonendfinally
detecting nonendHaman
detecting nonendalways
detecting nonendthe
detecting nonendjust
detecting nonendJust
detecting nonendthe
detecting nonendthe
detecting nonend,
detecting nonendasked
detecting nonend,
detecting nonend,
detecting nonend,
detecting nonendlike
detecting nonend,
detecting nonend,
detecting nonendHaman
detecting nonendevil
detecting nonendclearly
detecting nonend7
detecting nonendpromise
detecting nonendwill
detecting nonendeliminate
detecting nonend,
detecting nonendfall
detecting nonendanother
detecting nonend,
detecting nonendhated
detecting nonend,
detecting nonend,
detecting nonendJustice
detecting nonendcharacter
detecting nonend,
detecting nonendWe
detecting nonendevil
detecting nonend,
detecting nonendjust
detecting nonendgiven
detecting nonend,
detecting nonend,
detecting nonend,
detecting nonend,


[{'text': "Take your copy of God's word this morning. Please find Esther chapter number 7 The Book of Esther chapter number 7. Every great story has an antagonist it has a villain who is either working behind the scenes unknown to most of the characters in the story or in some situations he is working openly and what he's doing is known to all of the characters in the story.",
  'start_time': 0.3,
  'end_time': 29.9},
 {'text': "Tory. Now if you're like me, whenever you watch a good movie or you read a good story, you try to figure out who the villain is as quickly as possible, and then when you figure out who the villain is, you then spend the rest of your time, hoping for and routing on the people to expose the enemy and make him or her pay for their misdeeds. And if the story is really, really good, then you cheer when Justice is finally Fenced, it doesn't happen very often in movies. But back a while ago, I went and saw a series of movies, the Avenger movies. And of course you saw 

In [69]:
#" ".join([iterator.word for iterator in response.results[0].alternatives[0].words])
results = [result for result in response.results]
transcripts = []
words = []
for result in results:
    words_results.append(result.alternatives[0])

for transcript in transcripts:
    words.append(transcript.words)
print(words_results)
#" ".join([w.words.word for w in words_results])

t


In [129]:
#response.results[0].alternatives[0].words

nlp = spacy.load("en_core_web_sm")
doc = nlp(" ".join([iterator.word for iterator in response.results[0].alternatives[0].words]))
sentences = []
sentence = ""

for i, token in enumerate(doc):
    sentence += f" {token}"
    if token.is_sent_end:
      sentences.append(sentence)
      sentence = ""

print(doc)
print(sentences)

Take your copy of God's word this morning. Please find Esther chapter number 7 The Book of Esther chapter number 7. Every great story has an antagonist it has a villain who is either working behind the scenes unknown to most of the characters in the story or in some situations he is working openly and what he's doing is known to all of the characters in the story.
[" Take your copy of God 's word this morning .", ' Please find Esther chapter number 7 The Book of Esther chapter number 7 .', " Every great story has an antagonist it has a villain who is either working behind the scenes unknown to most of the characters in the story or in some situations he is working openly and what he 's doing is known to all of the characters in the story ."]


In [132]:
new_sentences = []
for result in response.results:
    alternatives = result.alternatives
    
    for alternative in alternatives:
        sentence = alternative.transcript.strip()
        if sentence:
            new_sentences.append({
                'text': sentence,
                'start_time': alternative.words[0].start_time.total_seconds(),
                'end_time': alternative.words[-1].end_time.total_seconds()
            })

In [135]:
def concatenate_sentences(transcriptions):
    nlp = spacy.load('en_core_web_sm')
    complete_sentences = []

    for transcription in transcriptions:
        text = transcription['text']
        start_time = transcription['start_time']
        end_time = transcription['end_time']

        doc = nlp(text)
        sentences = [sent.text for sent in doc.sents]

        if not sentences:
            continue

        last_sentence = sentences[0]
        for sentence in sentences[1:]:
            if last_sentence[-1] not in ('.', '!', '?') and sentence[0].islower():
                last_sentence += ' ' + sentence
            else:
                complete_sentences.append({
                    'text': last_sentence.strip(),
                    'start_time': start_time,
                    'end_time': end_time
                })
                last_sentence = sentence

        complete_sentences.append({
            'text': last_sentence.strip(),
            'start_time': start_time,
            'end_time': end_time
        })

    return complete_sentences

manipulated_sentences = concatenate_sentences(new_sentences)
print(manipulated_sentences)

[{'text': "Take your copy of God's word this morning.", 'start_time': 0.3, 'end_time': 29.9}, {'text': 'Please find Esther chapter number 7 The Book of Esther chapter number 7.', 'start_time': 0.3, 'end_time': 29.9}, {'text': "Every great story has an antagonist it has a villain who is either working behind the scenes unknown to most of the characters in the story or in some situations he is working openly and what he's doing is known to all of the characters in the story.", 'start_time': 0.3, 'end_time': 29.9}, {'text': 'Tory.', 'start_time': 30.0, 'end_time': 59.9}, {'text': "Now if you're like me, whenever you watch a good movie or you read a good story, you try to figure out who the villain is as quickly as possible, and then when you figure out who the villain is, you then spend the rest of your time, hoping for and routing on the people to expose the enemy and make him or her pay for their misdeeds.", 'start_time': 30.0, 'end_time': 59.9}, {'text': 'And if the story is really, re